In [1]:
import numpy as np
from qiskit.quantum_info import partial_trace as ptrace
from qiskit.quantum_info import random_unitary
from numpy import random


In [13]:
def Shadow(rho):
    dim = len(rho)
    qubit_num = int(np.log2(len(rho)))
    U = np.array(random_unitary(dim))
    
    evoluted_rho = np.dot(U,np.dot(rho,np.conj(U).T))
    prob_lst = [0 for i in range(dim)]
    for i in range(dim):
        if abs(evoluted_rho[i][i]) > 10**(-10):
            prob_lst[i] = evoluted_rho[i][i]
        else: 
            prob_lst[i] = 0
    measurement_res = np.random.choice(np.arange(0,dim),p = prob_lst)
    
    rho_hat = np.array([[0 for i in range(dim)] for j in range(dim)])
    rho_hat[measurement_res][measurement_res] = 1
    rho_hat = (dim+1)*np.dot(np.conj(U).T,np.dot(rho_hat,U))-np.identity(dim)
    return rho_hat

def Shadow_estimator(rho,obs,M):
    res = 0
    for i in range(M):
        res += np.trace(np.dot(obs,Shadow(rho)))/M
    return res

def variance(rho,obs,M,times):
    var = 0
    real_value = np.trace(np.dot(obs,rho))
    for i in range(times):
        estimator = Shadow_estimator(rho,obs,M)
        var += ((estimator-real_value)**2).real/times
    return var

def H_Shadow(rho):
    n = int(np.log2(len(rho)))
    Hadamard = np.array([[1,1],[1,-1]]/np.sqrt(2))
    for i in range(n-1):
        Hadamard = np.kron(Hadamard,np.array([[1,1],[1,-1]]/np.sqrt(2)))
    RDU = np.diag([np.e**(complex(0,2*np.pi*random.rand())) for i in range(2**n)])
    U = np.dot(Hadamard,RDU)
    
    evolve = np.dot(U,np.dot(rho,np.conj(U).T))
    prob_lst = [evolve[i][i] for i in range(2**n)]
    measurement_res = int(np.random.choice(np.arange(0,2**n),p = prob_lst))
    
    snap = np.zeros((2**n,2**n))
    snap[measurement_res][measurement_res] = 1
    snap = np.dot(np.conj(U).T,np.dot(snap,U))
    snap = 2**n*snap-np.identity(2**n)
    
    return snap

def H_Shadow_estimator(rho,obs,M):
    res = 0
    for i in range(M):
        res += np.trace(np.dot(obs,H_Shadow(rho)))/M
    return res

def D_variance(rho,obs,M,times):
    var = 0
    real_value = np.trace(np.dot(obs,rho))
    for i in range(times):
        estimator = H_Shadow_estimator(rho,obs,M)
        var += ((estimator-real_value)**2).real/times
    return var

In [ ]:
M=10
times = 1000
X = np.array([[0,1],[1,0]])
S_variance_lst = []
DS_variance_lst = []
for n in range(1,10):
    U1 = random_unitary(2**n)
    U2 = random_unitary(2**n)

    rho0 = np.diag([1]+[0 for i in range(2**n-1)])
    rho = np.dot(U1,np.dot(rho0,np.conj(U1).T))
    obs = X
    for i in range(n-1):
        obs = np.kron(X,obs)
    S_variance_lst.append(variance(rho,obs,M,times))
    DS_variance_lst.append(D_variance(rho,obs,M,times))
    
    print('n=',n)
    print('S:',S_variance_lst[n-1])
    print('D:',DS_variance_lst[n-1])

In [15]:
print(S_variance_lst)
print(DS_variance_lst)

[0.2766646506914531, 0.4792013913995822, 0.8570296581172637, 1.7351877854861233, 3.191508420947368, 6.220622693484177, 12.04967458147364, 27.39596927312283]
[0.18478720747259503, 0.4006597318297908, 0.7940089643022166, 1.4999795229434008, 3.219958258638862, 6.781304359618185, 13.290053469117396, 25.236019475069696]
